# Fully Convolutional Network & U-Net

contributors:

    -Skander Jemaa

In [ ]:
%matplotlib inline
import importlib
from scipy.misc import imread, imresize, imsave, fromimage, toimage
from scipy.optimize import fmin_l_bfgs_b
import numpy as np
import time
import random
from PIL import Image
import os
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, AveragePooling2D, MaxPooling2D
from keras.layers import Conv2D, UpSampling2D, Lambda, ZeroPadding2D
from keras.models import Sequential
from vgg16_avg import VGG16_Avg
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.transform as trans
import random as r
from keras.optimizers import Adam

# Example of Image

In [ ]:
file ='images/Spongesbob/img/frame_0001.png'

img = io.imread(file)/255
greyscale = rgb2grey(img)
canny = feature.canny(greyscale)

plt.figure(figsize=(15,10))

plt.subplot(131)
plt.title('Input')
plt.imshow(greyscale,cmap = 'gray')

plt.subplot(132)
plt.title('Ground Truth')
skimage.io.imshow(img)

plt.subplot(133)
plt.title('Prediction')
plt.imshow(canny,cmap = 'gray')

In [ ]:
np.array(greyscale).shape

# Data loading and Prepocessing

In [ ]:
def create_data(src, mask, label=False):
    files = glob(src + mask, recursive=True)
    imgs = []
    imgs_grey = []
    for file in files:
        feat_grey = []
        img = io.imread(file)
        if label:
            img[img == 4] = 1
            img[img != 1] = 0
            img = img.astype('float32')
        else:
            img = img/255
        imgs.append(img)
        img_grey = rgb2grey(img)
        feat_grey.append(img_grey)
        canny = feature.canny(img_grey)
        feat_grey.append(canny)
        imgs_grey.append(feat_grey)
    name = 'y'
    np.save(name, np.array(imgs).astype('float32'))
    print('Saved', len(files), 'to', name)
    name = 'x'
    np.save(name, np.array(imgs_grey)[..., np.newaxis].astype('float32'))
    print('Saved', len(files), 'to', name)

In [ ]:
create_data('images/Spongebob/img/', '**.png')

In [ ]:
x = np.load('x.npy')
print('x: ', x.shape)
y = np.load('y.npy')
print('y:', y.shape)

# Models

## Fully Convolutional Network

In [ ]:
def colorization_net(arr):
    inputs = Input(arr.shape[1:])
    conv1 = Conv2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = Conv2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Conv2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Conv2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Conv2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    up1 = UpSampling2D(size=(2, 2))(pool4)
    conv5 = Convolution2D(128, 2, 2, activation='relu', border_mode='same')(up1)
    
    up2 = UpSampling2D(size=(2, 2))(conv5)
    conv6 = Convolution2D(64, 2, 2, activation='relu', border_mode='same')(up2)
    
    up3 = UpSampling2D(size=(2, 2))(conv6)
    conv7 = Convolution2D(33, 2, 2, activation='relu', border_mode='same')(up3)

    up4 = UpSampling2D(size=(2, 2))(conv7)
    conv8 = Convolution2D(3, 2, 2, activation='relu', border_mode='same')(up4)
    
    outputs = conv8
    
    return outputs #model

# UNet

In [ ]:
def conv_block(x, filters, size, stride=(2,2), mode='same', act=True):
    x = Convolution2D(filters, size, size, subsample=stride, border_mode=mode)(x)
    x = BatchNormalization(mode=2)(x)
    return Activation('relu')(x) if act else x

def res_block(ip, nf=64):
    x = conv_block(ip, nf, 3, (1,1))
    x = conv_block(x, nf, 3, (1,1), act=False)
    return merge([x, ip], mode='sum')

In [ ]:
from keras.models import Input, Model
from keras.layers import Conv2D, Concatenate, MaxPooling2D, Reshape
from keras.layers import UpSampling2D, Activation, Permute

def level_block(m, dim, depth, factor, acti):
    if depth > 0:
        n = Conv2D(dim, 3, activation=acti, padding='same')(m)
        n = Conv2D(dim, 3, activation=acti, padding='same')(n)
        m = MaxPooling2D()(n)
        m = level_block(m, int(factor*dim), depth-1, factor, acti)
        m = UpSampling2D()(m)
        m = Conv2D(dim, 2, activation=acti, padding='same')(m)
        m = Concatenate(axis=3)([n, m])
    m = Conv2D(dim, 3, activation=acti, padding='same')(m)
    return Conv2D(dim, 3, activation=acti, padding='same')(m)

def UNet(img_shape, n_out=1, dim=64, depth=4, factor=2, acti='relu', flatten=False):
    i = Input(shape=img_shape)
    o = level_block(i, dim, depth, factor, acti)
    o = Conv2D(n_out, (1, 1))(o)
    if flatten:
        o = Reshape(n_out, img_shape[0] * img_shape[1])(o)
        o = Permute((2, 1))(o)
    o = Activation('relu')(o)
    return o

# Training 

## Fully Convolutionnal

In [ ]:
model = colorization_net(x)
model.compile(optimizer=Adam(lr=0.01), loss='mse')
model.fit(x, y, validation_split=0.2, epochs=50, batch_size=8)

## U-Net

In [ ]:
model_u = UNet(x.shape[1:], n_out = 3, dim=8, factor=1)
model_u.load_weights('unet.h5')

In [ ]:
model_u.compile(optimizer=Adam(lr=0.001), loss='mae')

In [ ]:
model_u.fit(x[:500], y[:500], validation_split=0.2, epochs=100, batch_size=8)

In [ ]:
model_u.compile(optimizer=Adam(lr=0.0001), loss='mse')
model_u.fit(x[:500], y[:500], validation_split=0.2, epochs=100, batch_size=8)

In [ ]:
model_u.save_weights('unet.h5')

# Results

## Fully Convolutional

In [ ]:
pred = model.predict(x[:50])
pred = np.clip(pred,0,255)/255

In [ ]:
import random as r
for n in range(20):
    i = int(r.random() * pred.shape[0])
    plt.figure(figsize=(15,10))

    plt.subplot(131)
    plt.title('Input')
    plt.imshow(x[i,:,:,0],cmap = 'gray')

    plt.subplot(132)
    plt.title('Ground Truth')
    skimage.io.imshow(y[i]/255)

    plt.subplot(133)
    plt.title('Prediction')
    plt.imshow(pred[i])

    plt.show()

## U-Net

In [ ]:
pred_u = model_u.predict(x[:50])
pred_u = np.clip(pred_u,0,255)/255

In [ ]:
import random as r
import skimage
for n in range(20):
    i = int(r.random() * pred_u.shape[0])
    plt.figure(figsize=(15,10))

    plt.subplot(131)
    plt.title('Input')
    plt.imshow(x[i,:,:,0],cmap = 'gray')

    plt.subplot(132)
    plt.title('Ground Truth')
    skimage.io.imshow(y[i])

    plt.subplot(133)
    plt.title('Prediction')
    plt.imshow(pred_u[i])

    plt.show()